# SIATA AUTOMATION DOWNLOAD SYSTEM (TRANSFORM)
<b>By: <em>David Serna</em></b><br>
<b><em>Data Scientist and Data Engineer</em></b>

If you want to know more about the project, what is SIATA and the motivations to make this repository, please [visit](https://github.com/dsernag/siata_automation) it

Also, here is my LinkedIn profile: [<img width="25px" src="../imgs/linkedIn_PNG32.png" alt="LinkedinLogo"/>](https://www.linkedin.com/in/dsernag/)

This is open source code, so you can use it, but remember to be etic and if is possibly, cite it.©

## Configure your environment

I highly recommend to create an isolated Python environment, for sanity check, replication purposes and as a learning activity. Also is required to install wget for Windows users.

### Conda environment

I love [miniconda](https://docs.conda.io/en/latest/miniconda.html), is light and portable and allow an easly manipulation of environments. If you don't install conda, please at least create an independt environment in Python, here is the [documentation](https://docs.python.org/3/library/venv.html).

After install miniconda, create a environment this way (This command is valid for any operating system):

```bash
conda create -n siata python=3.9 selenium ipykernel beautifulsoup4 pandas numpy matplotlib seaborn mysql-connector-python
```

There is also a `requirements.txt` file to install via pip

Now we are ready to begin!🚀🚀🚀

<b>NOTE:</b> This notebook is not intended no be used in a row. Is intended to be executed line by line, with supervision and understanding the process.


## Transformation process

### Import libraries and explore data

In [1]:
import pandas as pd
import os
CSVFILES = list(map(lambda x: "../data/air/" + x ,os.listdir("../data/air/")))

Because I've had working with SIATA data, I know in advance that not every csv file had the same schema of others. So, our first step is to check if all datasets have the same column lenght and if so, verify that all have the same column names:

In [2]:
# Check column lenght and column names

# This function check columns in general, length and names
def check_columns():
    set_column_lenght = set()
    set_column_names = set()
    for file in CSVFILES:
        file_df = pd.read_csv(file, encoding="utf-8")
        column_length =  file_df.shape[1]
        column_names = file_df.columns.tolist()
        set_column_lenght.add(column_length)
        set_column_names.add("__".join(column_names))
    return set_column_lenght, set_column_names
column_lenght, column_names = check_columns()
print(column_lenght)
print(column_names)
print(len(column_names))

{36}
{'Fecha_Hora__codigoSerial__pm25__calidad_pm25__pm10__calidad_pm10__pm1__calidad_pm1__no__calidad_no__no2__calidad_no2__nox__calidad_nox__ozono__calidad_ozono__co__calidad_co__so2__calidad_so2__pst__calidad_pst__dviento_ssr__calidad_dviento_ssr__haire10_ssr__calidad_haire10_ssr__p_ssr__calidad_p_ssr__pliquida_ssr__calidad_pliquida_ssr__rglobal_ssr__calidad_rglobal_ssr__taire10_ssr__calidad_taire10_ssr__vviento_ssr__calidad_vviento_ssr', 'Unnamed: 0__codigoSerial__pm25__calidad_pm25__pm10__calidad_pm10__pm1__calidad_pm1__no__calidad_no__no2__calidad_no2__nox__calidad_nox__ozono__calidad_ozono__co__calidad_co__so2__calidad_so2__pst__calidad_pst__dviento_ssr__calidad_dviento_ssr__haire10_ssr__calidad_haire10_ssr__p_ssr__calidad_p_ssr__pliquida_ssr__calidad_pliquida_ssr__rglobal_ssr__calidad_rglobal_ssr__taire10_ssr__calidad_taire10_ssr__vviento_ssr__calidad_vviento_ssr'}
2


Fortunately the whole stand of csv files have 36 columns, however there are 2 patterns of column names. This might be problematic, so let's explore the maladjustment. Aparently the only difference is in the first column:

In [3]:
# Split the pattern create before
pattern1 = list(column_names)[0].split("__")
pattern2 = list(column_names)[1].split("__")

# Drop the first element
pattern1.pop(0)
pattern2.pop(0)

# Rejoin the patterns, and compare it
string_pattern1 = "__".join(pattern1)
string_pattern2 = "__".join(pattern2)
string_pattern1 == string_pattern2

True

Indeed the supposition  was right. Now let's check how many datafiles have the first column as `Unnamed: 0`

As we can observe, is due the fact the first column has no header

<img src="../imgs/example-csv.PNG" width="800px">

In [4]:
# Check how many csv files have the issue of missing header
def check_integrity():
    count = 0
    for file in CSVFILES:
        file_df = pd.read_csv(file, encoding="utf-8")
        if file_df.columns[0] == "Unnamed: 0":
            count += 1
    return count
count = check_integrity()
print(f"There are {count} files with no first column header, this is the {count*100/len(CSVFILES):.2f}% of the files")

There are 669 files with no first column header, this is the 46.23% of the files


After we know the issue, let's declare a variable with the columns names. But, before make this process slower than it is, we are going to use the first 3 columns, because we are only interested in `pm25` and `calidad_pm25`. The list of columns will be:

In [5]:
valid_columns = ['date', 'station', 'pm25', 'quality_index']

The next function will produce a single `pd.DataFrame` object with the whole data. However, to improve performance, we also going to validate the data. How?

[Here](../data/Generalidades_Info_Aire.pdf) is the documentation about the air quality stations. This table shows what means `quality_index`:

<img src="../imgs/data_quality.PNG" width="500px">

So we are going to drop, any record where `quality_index` is -1 or above 2.5, also any row where `pm25` is -9999 and `quality_index` 1. Let's pute in code!

### Transform and save stage data

In [6]:
# Return a single data set filtered
def transform():
    df_transformed = pd.DataFrame(columns = valid_columns) 
    for file in CSVFILES:
        file_df = pd.read_csv(file, encoding = "utf-8", usecols = [i for i in range(0, 4)], names = valid_columns, header = None, skiprows = 1)
        conditions = (file_df['quality_index'] == -1) | (file_df['quality_index'] > 2.5) | (file_df['pm25'] < 0) | (file_df['pm25'] == -9999)
        file_df_transormed = file_df[~conditions]
        df_transformed = pd.concat([df_transformed, file_df_transormed], axis = 0)
    df_transformed['date'] = pd.to_datetime(df_transformed['date'])
    return df_transformed.reset_index(drop = True)

df_transformed = transform()

In [7]:
df_transformed.head()

,date,station,pm25,quality_index
0,2019-06-01 00:00:00,38,12.0,1.0
1,2019-06-01 01:00:00,38,9.0,1.0
2,2019-06-01 02:00:00,38,10.0,1.0
3,2019-06-01 03:00:00,38,6.0,1.0
4,2019-06-01 04:00:00,38,1.0,2.1


Yeah! We did it!. Let's save our final file in a csv:

In [8]:
df_transformed.to_csv("../data/stage/1605-2208-historic-air.csv", index = False)

Great folks! It was easy right? Remember that we've made a short preprocessing. Is the job of Data Scientist to do a proper sanity check of the data! Let's put this csv file into a proper RDBMS! [Continue](Load.ipynb) to the Load phase.

This is open source code, so you can use it, but remember to be etic and if is possibly, cite it.©